# **Search a Job using the resume**

## **Introduction**

In this notebook, we'll learn Question answering using embeddings-based search.
- First, we will prepare some job descriptions and their embeddings
- We will also prepare some resumes and their embeddings
- We will have a query, which will then match to appropriate job description
- Based on the matched job description, we will use OpenAI's prompts to ask questions


In [ ]:
!pip install openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 226.7/226.7 kB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.8/77.8 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 5.4 MB/s eta 0:00:00


## **Get open AI key**

In [ ]:
from google.colab import userdata
open_ai_key = userdata.get('OPEN_AI_KEY')


## **Setting Up Our Environment**

Tools that are used are:

**openai:**  It is a Python library for Open AI APIs


**numpy(np)** It is a fundamental package for scientific computing in Python.


In [ ]:
from openai import OpenAI
import numpy as np
client = OpenAI(api_key=open_ai_key)

### **Sample Job Description for Designer**




In [ ]:
designer_doc ="""Title: Designer
Overview: The Designer role involves creating visual designs aligned with the company's brand.
Responsibilities include developing digital and print materials, adhering to brand guidelines, collaborating with teams, staying updated on design trends, managing multiple projects, providing feedback,
and maintaining organized files.
Qualifications: Bachelor's degree in Graphic Design or related field, experience in graphic design, proficiency in Adobe Creative Suite,
strong communication skills, attention to detail, ability to work independently, familiarity with web design principles and motion graphics is a plus.
"""

### **Sample Job Description for Python Developer**



In [ ]:
python_doc = """
Title: Python Developer
Overview: The Python Developer role involves designing, implementing, and maintaining Python-based applications and software solutions. Responsibilities include writing efficient, reusable code,
collaborating with cross-functional teams, troubleshooting technical issues, and staying updated on Python programming language and related technologies.
Familiarity with front-end technologies including HTML, CSS, and JavaScript. Knowledge of relational databases (e.g., MySQL, PostgreSQL) and ORM libraries (e.g., SQLAlchemy)
Experience with version control systems such as Git. Familiarity with software development methodologies like Agile and Scrum
Qualifications: Bachelor's degree in Computer Science or related field, proven experience as a Python Developer, proficiency in Python programming and frameworks (e.g., Django, Flask),
strong problem-solving skills, familiarity with database systems, front-end technologies, and cloud platforms is a plus.
"""

### **Sample Job Description for JavaScript Developer**

In [ ]:
javascript_doc ="""
Title: JavaScript Developer
Overview: The JavaScript Developer is responsible for designing, developing, and implementing web-based applications and user interfaces using JavaScript, HTML, and CSS.
This role involves collaborating with cross-functional teams to deliver high-quality software solutions that meet project requirements and industry standards.
Responsibilities:
    Develop and maintain web applications using JavaScript, HTML, and CSS.
    Implement user interfaces and interactive features to enhance user experience.
    Collaborate with designers and backend developers to integrate front-end components with server-side logic.
"""

### **Sample Resume of a Python Developer**



In [ ]:
resume_1 = """
Objective:
Experienced Python developer with a passion for creating efficient, scalable, and maintainable software solutions. Seeking to leverage expertise in Python programming,
software development methodologies, and problem-solving skills in a challenging and dynamic work environment.

Education:

    Bachelor of Science in Computer Science
    XYZ University, City, State
    Graduated: May 20XX

Skills:

    Proficient in Python programming language
    Strong understanding of data structures and algorithms
    Proficiency in Python programming and frameworks (e.g., Django, Flask),
    Familiarity with front-end technologies including HTML, CSS, and JavaScript
    Knowledge of relational databases (e.g., MySQL, PostgreSQL) and ORM libraries (e.g., SQLAlchemy)
    Experience with version control systems such as Git
    Familiarity with software development methodologies like Agile and Scrum
    Excellent problem-solving and debugging skills
    Strong communication and collaboration abilities

Professional Experience:

Software Developer | ABC Company
City, State | Dates
    Worked closely with front-end developers to integrate user-facing elements with server-side logic
    Collaborated with cross-functional teams to deliver high-quality software solutions within project timelines

    """

## **Create Document Embedding**

In [ ]:
def get_embedding(text, model="text-embedding-3-small"):
   text = text.replace("\n", " ")
   return client.embeddings.create(input = [text], model=model).data[0].embedding

### **Embeddings of all job descriptions and resume**

In [ ]:
designer_doc_embedding = get_embedding(designer_doc)
python_doc_embedding = get_embedding(python_doc)
javascript_doc_embedding = get_embedding(javascript_doc)

print("designer doc embedding" , designer_doc_embedding)
print("python doc embedding" , python_doc_embedding)
print("javascript doc embedding" , javascript_doc_embedding)

designer doc embedding [-0.0035818414762616158, 0.04810680076479912, 0.042654987424612045, -0.014316461980342865, -0.026016054674983025, -0.020793216302990913, 0.037770163267850876, 0.02974509447813034, -0.04775788635015488, 0.061932601034641266, 0.004023438319563866, 0.01145971193909645, -0.046885594725608826, -0.03687606751918793, 0.017423996701836586, 0.06869284808635712, -0.03744305670261383, -0.012004893273115158, 0.020258938893675804, 0.028720153495669365, 0.03984185308218002, 0.013902124017477036, -0.00234427978284657, 0.050069455057382584, 0.007681605406105518, 0.021316591650247574, -0.005277355667203665, 0.032361965626478195, 0.009573384188115597, -0.01032028254121542, 0.016911525279283524, -0.01927761174738407, -0.015210559591650963, -0.031664133071899414, 0.018198153004050255, -0.007817900739610195, 0.00884284172207117, -0.012386520393192768, -0.01152513362467289, -0.007812448777258396, -0.020095383748412132, 0.01495977584272623, 0.020531529560685158, -0.01768568344414234, 0

## **Get the related docs from the question**

In [ ]:
def get_related_description(query):
    query_vector = get_embedding(query)

    titles = ["designer", "python developer", "javascript developer"]
    all_desc_embeddings = [designer_doc_embedding, python_doc_embedding, javascript_doc_embedding]
    similarity_scores = [np.dot(query_vector, embedding).round(2) for embedding in all_desc_embeddings]
    descriptions = [designer_doc, python_doc, javascript_doc]
    max_score = max(similarity_scores)

    if all(score < 0.7 for score in similarity_scores):
        print("We couldn't find a job description related to your query\n")
        return

    related_title = None
    related_description = None
    for title, score, desc in zip(titles, similarity_scores, descriptions):
        print(f"question and {title} similarity_score \n", score)
        if max_score == score:
            related_title = title
            related_description = desc

    print(f"Your query is related to the {related_title}'s job description\n")

    return related_description

In [ ]:
description = get_related_description(resume_1)

question and designer similarity_score 
 0.36
question and python developer similarity_score 
 0.71
question and javascript developer similarity_score 
 0.49
Your query is related to the python developer's job description



## **Query llm according to related docs**

In [ ]:
def query_llm(ques, doc):
  system_prompt=f"""I have a document below. I want you to answer my question using it.
    Document: {doc}
    """
  user_prompt= f"My Question is: {ques}"
  prompt_answer_response = client.chat.completions.create(
  model="gpt-3.5-turbo",
  messages=[
    {"role": "system", "content":system_prompt},
    {"role": "user", "content":  user_prompt}
  ]
  )

  return prompt_answer_response

In [ ]:
resp = query_llm("describe the position", description)
print(resp.choices[0].message.content)

The Python Developer position involves designing, implementing, and maintaining Python-based applications and software solutions. Responsibilities include writing efficient and reusable code, collaborating with cross-functional teams, troubleshooting technical issues, and staying updated on Python programming language and related technologies. Additionally, familiarity with front-end technologies like HTML, CSS, and JavaScript, knowledge of relational databases (e.g., MySQL, PostgreSQL) and ORM libraries (e.g., SQLAlchemy), experience with version control systems such as Git, and familiarity with software development methodologies like Agile and Scrum are required. The qualifications for the position include a Bachelor's degree in Computer Science or related field, proven experience as a Python Developer, proficiency in Python programming and frameworks like Django and Flask, strong problem-solving skills, and familiarity with database systems, front-end technologies, and cloud platfor